In [2]:
!pip install transformers
!pip install torch torchvision
!pip install pandas
!pip install scikit-learn
!pip install sentencepiece
!pip install sacremoses
!pip install evaluate
!pip install datasets

/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "


/Users/jayjoshi/.zshenv:3: unmatched "


In [3]:
# checkpoint_hindi = "Hate-speech-CNERG/hindi-abusive-MuRIL"
# checkpoint_spanish = "Hate-speech-CNERG/dehatebert-mono-spanish"
# checkpoint_english = "Hate-speech-CNERG/dehatebert-mono-english"
# checkpoint_xlm_r = "FineTuned_Model/xlm_r/"
# checkpoint = "Hate-speech-CNERG/hindi-abusive-MuRIL"

In [4]:
import torch
import pandas as pd
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import datasets

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [5]:
import evaluate

metric_accuracy = evaluate.load("accuracy")
metric_glue = evaluate.load("glue", "mrpc")
metric_precision = evaluate.load("precision")
metric_recall = evaluate.load("recall")
metric_rocauc = evaluate.load("roc_auc")

def compute_metrics(labels):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
    score_dict["accuracy"] = metric_accuracy.compute(predictions=preds, references=labels)
    score_dict["glue"] = metric_glue.compute(predictions=preds, references=labels)
    score_dict["precision"] = metric_precision.compute(predictions=preds, references=labels)
    score_dict["recall"] = metric_recall.compute(predictions=preds, references=labels)
#     score_dict["rocauc"] = metric_rocauc.compute(predictions=preds, references=labels)

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

In [6]:
model_names=['HI-EN-ES', 'HI-EN', 'HI-ES', 'EN-ES', 'HI', 'ES', 'EN', 'en1']
# datasets = ["My_Data/en_sentence_templates.txt", "My_Data/es-es_sentence_templates.txt", "My_Data/hi-Latn_sentence_templates.txt"]
# datasets = ["test_data/hateval.csv", "test_data/R_36.csv"]
# datasets = [ "test_data/R_63.csv"]
datasets = [datasets.load_dataset("ethos", "binary")]
results = []
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

Found cached dataset ethos (/Users/jayjoshi/.cache/huggingface/datasets/ethos/binary/1.0.0/898d3d005459ee3ff80dbeec2f169c6b7ea13de31a08458193e27dec3dd9ae38)


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
import datasets

In [8]:
dataset = datasets.load_dataset("ethos", "binary")

Found cached dataset ethos (/Users/jayjoshi/.cache/huggingface/datasets/ethos/binary/1.0.0/898d3d005459ee3ff80dbeec2f169c6b7ea13de31a08458193e27dec3dd9ae38)


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
dataset['train'][0]

{'text': "You should know women's sports are a joke", 'label': 1}

In [13]:
texts = []
labels = []
for i in dataset['train']:
    texts.append(i['text'])
    labels.append(int(i['label']))
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.1)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.1)


train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

int

In [29]:
checkpoint = "results/xlm_r/EN-ES//"
model = model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
training_args = TrainingArguments(
            output_dir='./results/test_data/xyz',          # output directory
            num_train_epochs=3,              # total number of training epochs
            per_device_train_batch_size=16,  # batch size per device during training
            per_device_eval_batch_size=64,   # batch size for evaluation
            warmup_steps=500,                # number of warmup steps for learning rate scheduler
            weight_decay=0.01,               # strength of weight decay
            logging_dir='./logs/test_data/xyz' ,            # directory for storing logs
            logging_steps=10,
            # use_mps_device=True
        )

trainer = Trainer(
model=model,                         # the instantiated 🤗 Transformers model to be trained
args=training_args,                  # training arguments, defined above
train_dataset=train_dataset,         # training dataset
eval_dataset=val_dataset,            # evaluation dataset

        )
score_dict = {"accuracy": "none", "glue": "none", "precision": "none", "recall": "none"}

x = []

predictions = trainer.predict(train_dataset)
preds = np.argmax(predictions.predictions, axis=-1)
preds_list = list(preds)
compute_metrics(predictions.label_ids)
values = list(score_dict.values())
x.append(values[0]["accuracy"])
x.append(values[1]["f1"])
x.append(values[2]["precision"])
x.append(values[3]["recall"])


loading configuration file results/xlm_r/EN-ES//config.json
Model config XLMRobertaConfig {
  "_name_or_path": "results/xlm_r/EN-ES//",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file results/xlm_r/EN-ES//pytorch_model.bin
All model checkpoint weights were used when initializing XLMRobertaForSequ

In [30]:
x

[0.6868811881188119, 0.56, 0.7092511013215859, 0.46264367816091956]

In [10]:
for model_name in model_names:
    
    checkpoint = "results/xlm_r/" + model_name 
    # local_files_only=True (Parameter for .from_pretrained to specify taking the tokenizer from local files / .cache)

    model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
    
    for dataset in datasets:
        ds = pd.DataFrame(dataset)
        ds.dropna(inplace=True)
#         ds.rename(columns={"phrase" : "Text", "toxicity": "Label"}, inplace=True)

        print(ds.head())
        texts = ds["text"].tolist()
#         labels =[0 if i == "nontoxic" else 1 for i in  ds["Label"].tolist()]
        labels = [int(i) for i in ds["label"].tolist()]

        train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.1)
        train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.1)

        
        train_encodings = tokenizer(train_texts, truncation=True, padding=True)
        val_encodings = tokenizer(val_texts, truncation=True, padding=True)
        test_encodings = tokenizer(test_texts, truncation=True, padding=True)

        train_dataset = IMDbDataset(train_encodings, train_labels)
        val_dataset = IMDbDataset(val_encodings, val_labels)
        test_dataset = IMDbDataset(test_encodings, test_labels)

        training_args = TrainingArguments(
            output_dir='./results/test_data/'+ model_name + "_" + dataset,          # output directory
            num_train_epochs=3,              # total number of training epochs
            per_device_train_batch_size=16,  # batch size per device during training
            per_device_eval_batch_size=64,   # batch size for evaluation
            warmup_steps=500,                # number of warmup steps for learning rate scheduler
            weight_decay=0.01,               # strength of weight decay
            logging_dir='./logs/test_data/' + model_name + "_" + dataset,            # directory for storing logs
            logging_steps=10,
            # use_mps_device=True
        )

        trainer = Trainer(
            model=model,                         # the instantiated 🤗 Transformers model to be trained
            args=training_args,                  # training arguments, defined above
            train_dataset=train_dataset,         # training dataset
            eval_dataset=val_dataset,            # evaluation dataset

        )

        score_dict = {"accuracy": "none", "glue": "none", "precision": "none", "recall": "none"}

        predictions = trainer.predict(train_dataset)
        preds = np.argmax(predictions.predictions, axis=-1)
        preds_list = list(preds)
        compute_metrics(predictions.label_ids)
        
        #Confusion Matrix
        test_positives_count = 0
        test_negatives_count = 0
        pred_poisitves_count = (preds==1).sum()
        pred_negatives_count = (preds==0).sum()
        
        true_positives_count = 0
        true_negatives_count = 0
        false_positives_count = 0
        false_negatives_count = 0
        for i in range(len(test_labels)):
            if train_labels[i] == 1:
                test_positives_count += 1
                if preds_list[i] == 1:
                    true_positives_count += 1
                else:
                    false_negatives_count += 1
            else:
                test_negatives_count += 1
                if preds_list[i] == 0:
                    true_negatives_count += 1
                else:
                    false_positives_count += 1
                    
        

        values = list(score_dict.values())
        x = []

        x.append(model_name)
        x.append(dataset)
        x.append(values[0]["accuracy"])
        x.append(values[1]["f1"])
        x.append(values[2]["precision"])
        x.append(values[3]["recall"])
        x.append((preds==1).sum())
        x.append((preds==0).sum())
        x.append(test_positives_count)
        x.append(test_negatives_count)
        x.append(true_positives_count)
        x.append(true_negatives_count)
        x.append(false_positives_count)
        x.append(false_negatives_count)
        
        results.append(x)



                                               train
0  {'text': 'You should know women's sports are a...
1  {'text': 'You look like Sloth with deeper Down...
2  {'text': 'You look like Russian and speak like...
3  {'text': 'Women deserve to be abused, I guess....
4  {'text': 'Women are made for making babies and...


KeyError: 'text'

In [64]:
# results=[['HI-EN-ES', 'My_Data/en_sentence_templates.txt', 0.5464557533539731, 0.24940623915886106, 0.7294723696534499, 0.1504168410210191, 6406, 55610], ['HI-EN-ES', 'My_Data/es-es_sentence_templates.txt', 0.6401228008029284, 0.5099212091976202, 0.7988714228133817, 0.37447451702801, 19848, 64842], ['HI-EN-ES', 'My_Data/hi-Latn_sentence_templates.txt', 0.4793261868300153, 0.08832807570977919, 0.43209876543209874, 0.049191848208011243, 1296, 20906], ['HI-EN', 'My_Data/en_sentence_templates.txt', 0.541827915376677, 0.22594529802767788, 0.7351533416060982, 0.13348569221360285, 5641, 56375], ['HI-EN', 'My_Data/es-es_sentence_templates.txt', 0.6017357421183138, 0.3734745054332683, 0.874706342991386, 0.23742383449057675, 11493, 73197], ['HI-EN', 'My_Data/hi-Latn_sentence_templates.txt', 0.4936041798036213, 0.0807783500940234, 0.5832349468713105, 0.043394237526352776, 847, 21355], ['HI-ES', 'My_Data/en_sentence_templates.txt', 0.6197432920536635, 0.5050996852046169, 0.7256829283000663, 0.38735635883735153, 16583, 45433], ['HI-ES', 'My_Data/es-es_sentence_templates.txt', 0.6655331207934821, 0.5779042737080526, 0.7829685859646289, 0.4579613622408011, 24766, 59924], ['HI-ES', 'My_Data/hi-Latn_sentence_templates.txt', 0.4877488514548239, 0.04740765558254461, 0.5099099099099099, 0.024859451862262825, 555, 21647], ['EN-ES', 'My_Data/en_sentence_templates.txt', 0.5744646542827657, 0.35047994093034707, 0.7445362333995608, 0.2291820903209193, 9563, 52453], ['EN-ES', 'My_Data/es-es_sentence_templates.txt', 0.6137914747904121, 0.432290762661853, 0.8154138292299633, 0.29410514382882247, 15272, 69418], ['EN-ES', 'My_Data/hi-Latn_sentence_templates.txt', 0.4877038104675254, 0.010268012530455969, 0.5462962962962963, 0.005182712579058327, 108, 22094], ['HI', 'My_Data/en_sentence_templates.txt', 0.7152186532507739, 0.7054683721628339, 0.7319698228128461, 0.6808188753339557, 28896, 33120], ['HI', 'My_Data/es-es_sentence_templates.txt', 0.6705041917581769, 0.6989719414448916, 0.6433465059475346, 0.7651268244296443, 50357, 34333], ['HI', 'My_Data/hi-Latn_sentence_templates.txt', 0.49635168002882624, 0.12229199372056518, 0.5744837758112095, 0.06842937456078707, 1356, 20846], ['ES', 'My_Data/en_sentence_templates.txt', 0.5861390608875129, 0.39866922824609907, 0.7325010761945759, 0.27385972253516594, 11615, 50401], ['ES', 'My_Data/es-es_sentence_templates.txt', 0.6323414806942969, 0.4815943259577444, 0.8161503301168106, 0.34157574039960326, 17721, 66969], ['ES', 'My_Data/hi-Latn_sentence_templates.txt', 0.4936942617782182, 0.03667837860999229, 0.7508771929824561, 0.018798313422347154, 285, 21917], ['EN', 'My_Data/en_sentence_templates.txt', 0.5406830495356038, 0.2487538571089485, 0.6884671532846716, 0.15180094634177746, 6850, 55166], ['EN', 'My_Data/es-es_sentence_templates.txt', 0.6057503837525091, 0.41159573530707555, 0.8108033048670416, 0.27580180435501395, 14403, 70287], ['EN', 'My_Data/hi-Latn_sentence_templates.txt', 0.49193766327357896, 0.02371473082915008, 0.8058823529411765, 0.012034434293745608, 170, 22032], ['en1', 'My_Data/en_sentence_templates.txt', 0.5459881320949432, 0.26726695466611144, 0.6977850251392852, 0.16528792609521356, 7359, 54657], ['en1', 'My_Data/es-es_sentence_templates.txt', 0.6016767032707522, 0.40746855898264595, 0.7949965729952022, 0.27393604458929666, 14590, 70100], ['en1', 'My_Data/hi-Latn_sentence_templates.txt', 0.4887397531753896, 0.011839470706015497, 0.6601941747572816, 0.0059732958538299364, 103, 22099]]

In [67]:
features = []
features.append("Model")
features.append("Dataset")
features.append("Accuracy")
features.append("F1")
features.append("Precision")
features.append("Recall")
features.append("Preds == 1")
features.append("Preds == 0")
features.append("Test_Pos_Count")
features.append("Test_Neg_Count")
features.append("True_Pos_Count")
features.append("True_Neg_Count")
features.append("False_Pos_Count")
features.append("False_Neg_Count")



In [68]:
print(features)
for i in results: 
    print(i)

['Model', 'Dataset', 'Accuracy', 'F1', 'Precision', 'Recall', 'Preds == 1', 'Preds == 0', 'Test_Pos_Count', 'Test_Neg_Count', 'True_Pos_Count', 'True_Neg_Count', 'False_Pos_Count', 'False_Neg_Count']
['HI-EN-ES', 'test_data/R_63.csv', 0.6237623762376238, 0.62, 0.8888888888888888, 0.4760076775431862, 279, 529, 67, 33, 33, 32, 1, 34]
['HI-EN', 'test_data/R_63.csv', 0.6175742574257426, 0.6171003717472119, 0.8706293706293706, 0.4779270633397313, 286, 522, 67, 33, 34, 28, 5, 33]
['HI-ES', 'test_data/R_63.csv', 0.6732673267326733, 0.7130434782608696, 0.8220551378446115, 0.6295585412667947, 399, 409, 67, 33, 40, 28, 5, 27]
['EN-ES', 'test_data/R_63.csv', 0.5829207920792079, 0.5548216644649934, 0.8898305084745762, 0.40307101727447214, 236, 572, 67, 33, 23, 29, 4, 44]
['HI', 'test_data/R_63.csv', 0.6844059405940595, 0.7312961011591147, 0.8107476635514018, 0.6660268714011516, 428, 380, 67, 33, 43, 25, 8, 24]
['ES', 'test_data/R_63.csv', 0.6014851485148515, 0.5985037406483791, 0.8540925266903915,

In [69]:
results_dict = dict()

In [70]:
zipped_results = list(zip(results))


In [71]:
import pandas as pd

df = pd.DataFrame(results)
df.to_csv("results5.csv",index=False, header=features)